In [1]:
import os
from experiment_setup import setups

In [2]:
# download the dataset, the links can be fund in the README
dataset_path = "../datasets/rees46"
model_path = "../trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [4]:
%run ../Preprocess/coveo_preproc.py --path $dataset_path

1566074 274797 11365
1464757 173480 11344
1463706 173480 10869
1463649 173423 10869
1463645 173423 10868
1463645 173423 10868
1463645 173423 10868


C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\coveo_preproc.py:177: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.min() / 1000).strftime(
C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\coveo_preproc.py:182: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.max() / 1000).strftime(
C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\coveo_preproc.py:177: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent d

                                             Dataset  NumEvents  NumSessions  \
0      coveo_ecommerce\coveo_processed_view_full.tsv    1463645       173423   
1      coveo_ecommerce\coveo_processed_view_test.tsv      52501         7748   
2  coveo_ecommerce\coveo_processed_view_train_ful...    1411113       165673   
3  coveo_ecommerce\coveo_processed_view_train_tr.tsv    1368003       159766   
4  coveo_ecommerce\coveo_processed_view_train_val...      43032         5905   

   NumItems    NumDays                   StartTime  \
0     10868  17.999833  2018-12-08 00:00:11.994000   
1      8230   0.998696  2018-12-25 00:01:50.223000   
2     10868  16.999566  2018-12-08 00:00:11.994000   
3     10868  15.999713  2018-12-08 00:00:11.994000   
4      8014   0.997503  2018-12-24 00:03:10.240000   

                      EndTime  AvgItemViews  MinSessionLength  \
0  2018-12-25 23:59:57.577000    134.674733                 2   
1  2018-12-25 23:59:57.577000      6.379222                 2   

C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\coveo_preproc.py:177: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.min() / 1000).strftime(
C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\coveo_preproc.py:182: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.max() / 1000).strftime(


## Use a specific setup for your dataset

In [3]:
params = setups["rees46"]["params_xe"]

In [4]:
train_path = os.path.join(dataset_path,"rees46_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,"rees46_processed_view_test.tsv")

In [5]:
def create_gru4rec_tensorflow_script(model_name, train_path, test_path, model_path, loss, final_act, layers, batch_size, dropout_p_hidden, learning_rate, hidden_act, n_epochs, m, decay, initial_accumulator):
        """Creates the train and test script.
        """
        s_train_full = (
                f"python ../GRU4Rec_TensorFlow/gru4rec_BP/main.py --train_path {train_path} "
                f"--test_path {test_path} --checkpoint_dir {model_path}/{model_name} --layer {1} --size {layers} "
                f"--epoch {n_epochs} --lr {learning_rate} --hidden_act {hidden_act} --final_act {final_act} "
                f"--loss {loss} --dropout {dropout_p_hidden} --batch_size {batch_size} "
                f"--initial_accumulator_value {initial_accumulator} --decay {decay}"
        )
        s_test_full = s_train_full + f" --train 0 --test {n_epochs-1} --m {m}"
        return s_train_full, s_test_full

In [6]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_epochs = 5
m = '1 5 10 20'

## Train & test the out-of-the-box model

In [7]:
train_script_oob, test_script_oob = create_gru4rec_tensorflow_script(model_name="gru4rec_tensorflow_oob", train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, final_act=final_act, layers=layers, batch_size=50, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, hidden_act=hidden_act, n_epochs=n_epochs, m=m, decay=0.96, initial_accumulator=1e-1)

### Train the out-of-the-box model

In [32]:
from model_tracker import track_training_C02_emissions

track_training_C02_emissions(train_script_oob, "gru4rec_tensorflow_oob", "diginetica")

[codecarbon WARNING @ 20:51:32] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 20:51:32] [setup] RAM Tracking...
[codecarbon INFO @ 20:51:32] [setup] GPU Tracking...
[codecarbon INFO @ 20:51:32] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:51:34] [setup] CPU Tracking...
[codecarbon WARNING @ 20:51:34] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 20:51:35] We saw that you have a 13th Gen Intel(R) Core(TM) i9-13900HX but we don't know it. Please contact us.
[codecarbon INFO @ 20:51:35] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i9-13900HX
[codecarbon INFO @ 20:51:35] >>> Tracker's metadata:
[codecarbon INFO @ 20:51:35]   Platform system: Windows-11-10.0.22631-SP0
[codecarbon INFO @ 20:51:35]   Python version: 3.12.3
[codecarbon INFO @ 20:51:35]   CodeCarbon version: 2.4.2
[codecarbon INFO @ 20:51:35]   Available RAM : 31.746 GB
[codecarbon INFO @ 20:51:35]   CPU count: 32
[codecarbon 

CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 4090 Laptop GPU.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 4090 Laptop GPU.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 4090 Laptop GPU.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 4090 Laptop GPU.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.


[codecarbon INFO @ 20:51:54] Energy consumed for RAM : 0.000050 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 20:51:54] Energy consumed for all GPUs : 0.000413 kWh. Total GPU Power : 99.09882924052263 W
[codecarbon INFO @ 20:51:54] Energy consumed for all CPUs : 0.000182 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:51:54] 0.000645 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:52:09] Energy consumed for RAM : 0.000098 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 20:52:09] Energy consumed for all GPUs : 0.000669 kWh. Total GPU Power : 62.966113901585366 W
[codecarbon INFO @ 20:52:09] Energy consumed for all CPUs : 0.000359 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:52:09] 0.001126 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:52:24] Energy consumed for RAM : 0.000146 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 20:52:24] Energy consumed for all GPUs : 0.000860 kWh. Total GPU Power : 47.2391967917

CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: Average carbon intensity during training was 374.46 gCO2/kWh at detected location: Santiago, Santiago Metropolitan, CL.
CarbonTracker: Average carbon intensity during training was 374.46 gCO2/kWh at detected location: Santiago, Santiago Metropolitan, CL.
CarbonTracker: Average carbon intensity during training was 374.46 gCO2/kWh at detected location: Santiago, Santiago Metropolitan, CL.
CarbonTracker: Average carbon intensity during training was 374.46 gCO2/kWh at detected location: Santiago, San

0.0006139305004492953

### Test the out-of-the-box model

In [33]:
print(test_script_oob)
os.system(test_script_oob)

python ../GRU4Rec_TensorFlow/gru4rec_BP/main.py --train_path ../datasets/diginetica\diginetica_processed_view_train_full.tsv --test_path ../datasets/diginetica\diginetica_processed_view_test.tsv --checkpoint_dir ../trained_models/gru4rec_tensorflow_oob --layer 1 --size 192 --epoch 5 --lr 0.1 --hidden_act tanh --final_act softmax --loss cross-entropy --dropout 0.15 --batch_size 50 --initial_accumulator_value 0.1 --decay 0.96 --train 0 --test 4 --m 1 5 10 20


0

## Train & test the minor fix model


In [8]:
train_script_minor, test_script_minor = create_gru4rec_tensorflow_script(model_name='gru4rec_tensorflow_minorfix', train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, hidden_act=hidden_act, n_epochs=n_epochs, m=m, decay=1.0, initial_accumulator=1e-12)

In [18]:
print(train_script_minor)

python ../GRU4Rec_TensorFlow/gru4rec_BP/main.py --train_path ../datasets/rees46\rees46_processed_view_train_full.tsv --test_path ../datasets/rees46\rees46_processed_view_test.tsv --checkpoint_dir ../trained_models/gru4rec_tensorflow_minorfix --layer 1 --size 512 --epoch 5 --lr 0.065 --hidden_act tanh --final_act softmax --loss cross-entropy --dropout 0.0 --batch_size 240 --initial_accumulator_value 1e-12 --decay 1.0


### Train the minor fix model

In [9]:
from model_tracker import track_training_C02_emissions

track_training_C02_emissions(train_script_minor, "gru4rec_tensorflow_minorfix", "rees46")

[codecarbon WARNING @ 18:05:07] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 18:05:07] [setup] RAM Tracking...
[codecarbon INFO @ 18:05:07] [setup] GPU Tracking...
[codecarbon INFO @ 18:05:07] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:05:08] [setup] CPU Tracking...
[codecarbon WARNING @ 18:05:08] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 18:05:09] We saw that you have a 13th Gen Intel(R) Core(TM) i9-13900HX but we don't know it. Please contact us.
[codecarbon INFO @ 18:05:09] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i9-13900HX
[codecarbon WARNING @ 18:05:09] Failed to retrieve gpu information
Traceback (most recent call last):
  File "c:\Users\EVILAB\AppData\Local\Programs\Python\Python312\Lib\site-packages\codecarbon\core\gpu.py", line 238, in get_gpu_details
    devices_info.append(gpu_device.get_gpu_details())
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 

CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 4090 Laptop GPU.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
GPU model does not support retrieval of power usages in NVML.
See issue: https://github.com/lfwa/carbontracker/issues/36


[codecarbon INFO @ 18:05:28] Energy consumed for RAM : 0.000051 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 18:05:29] Energy consumed for all GPUs : 0.000109 kWh. Total GPU Power : 25.46148451477875 W
[codecarbon INFO @ 18:05:29] Energy consumed for all CPUs : 0.000186 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:05:29] 0.000346 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:05:43] Energy consumed for RAM : 0.000099 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 18:05:44] Energy consumed for all GPUs : 0.000261 kWh. Total GPU Power : 37.53620421966679 W
[codecarbon INFO @ 18:05:44] Energy consumed for all CPUs : 0.000363 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:05:44] 0.000724 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:05:58] Energy consumed for RAM : 0.000148 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 18:05:59] Energy consumed for all GPUs : 0.000404 kWh. Total GPU Power : 35.21523581575

CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.


[codecarbon INFO @ 18:20:13] Energy consumed for RAM : 0.002917 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 18:20:14] Energy consumed for all GPUs : 0.026884 kWh. Total GPU Power : 27.800957506057408 W
[codecarbon INFO @ 18:20:14] Energy consumed for all CPUs : 0.010636 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:20:14] 0.040437 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:20:28] Energy consumed for RAM : 0.002965 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 18:20:29] Energy consumed for all GPUs : 0.027004 kWh. Total GPU Power : 29.501304608066693 W
[codecarbon INFO @ 18:20:29] Energy consumed for all CPUs : 0.010814 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:20:29] 0.040783 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:20:43] Energy consumed for RAM : 0.003014 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 18:20:44] Energy consumed for all GPUs : 0.027115 kWh. Total GPU Power : 27.604754181

...

CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.


[codecarbon INFO @ 20:20:15] Energy consumed for RAM : 0.026146 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 20:20:15] Energy consumed for all GPUs : 0.137984 kWh. Total GPU Power : 21.822149286664178 W
[codecarbon INFO @ 20:20:15] Energy consumed for all CPUs : 0.095649 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:20:15] 0.259779 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:20:30] Energy consumed for RAM : 0.026194 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 20:20:30] Energy consumed for all GPUs : 0.138114 kWh. Total GPU Power : 32.15073240856847 W
[codecarbon INFO @ 20:20:30] Energy consumed for all CPUs : 0.095826 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:20:30] 0.260134 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:20:45] Energy consumed for RAM : 0.026242 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 20:20:45] Energy consumed for all GPUs : 0.138168 kWh. Total GPU Power : 13.4272569697

CarbonTracker: CRITICAL - Traceback (most recent call last):
  File "c:\Users\EVILAB\AppData\Local\Programs\Python\Python312\Lib\site-packages\carbontracker\tracker.py", line 293, in epoch_end
    self.tracker.epoch_end()
  File "c:\Users\EVILAB\AppData\Local\Programs\Python\Python312\Lib\site-packages\carbontracker\tracker.py", line 154, in epoch_end
    self._log_epoch_measurements()
  File "c:\Users\EVILAB\AppData\Local\Programs\Python\Python312\Lib\site-packages\carbontracker\tracker.py", line 172, in _log_epoch_measurements
    power_avg = np.mean(comp.power_usages[-1], axis=0)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EVILAB\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py", line 3504, in mean
    return _methods._mean(a, axis=axis, dtype=dtype,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EVILAB\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py", line 102, in 

AttributeError: 'tuple' object has no attribute 'tb_frame'

CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.


### Test the minor fix model

In [36]:
print(test_script_minor)
os.system(test_script_minor)

python ../GRU4Rec_TensorFlow/gru4rec_BP/main.py --train_path ../datasets/diginetica\diginetica_processed_view_train_full.tsv --test_path ../datasets/diginetica\diginetica_processed_view_test.tsv --checkpoint_dir ../trained_models/gru4rec_tensorflow_minorfix --layer 1 --size 192 --epoch 5 --lr 0.1 --hidden_act tanh --final_act softmax --loss cross-entropy --dropout 0.15 --batch_size 128 --initial_accumulator_value 1e-12 --decay 1.0 --train 0 --test 4 --m 1 5 10 20


0